# Objectif d'optimisation — C5.2.3

**But** : optimiser deux pipelines ML du projet — **Sentiment** (text → TF-IDF/chi2/SVD → LR) et **Emotions** (SBERT embeddings → OneVsRest LR / SGD) — pour améliorer les métriques clés (AP_macro pour émotions, F1/accuracy pour sentiment), tout en restant réaliste vis-à-vis des ressources (RAM 16GB, GPU disponible).

**Critères d'évaluation utilisés :**
- Emotions : **Average Precision (AP) par label** et **AP_macro** ; seuils optimisés par label (max F1) ; métriques globales micro/macro F1.
- Sentiment : **F1_macro**, **accuracy** et matrice de confusion.
- Comparaison : tableau récapitulatif (baseline vs optimisé) et calcul du **gain absolu** et **relatif (%)** sur les métriques principales.
- Traçabilité : sauvegarder toutes les expérimentations, modèles et paramètres dans `artifacts/`.

# Leviers d'optimisation

**DATA**
- Augmenter la taille d'entraînement (ex. pseudo-labels emotions → tout le dataset).
- Filtrer/normaliser les labels rares (seuils min support) ou regrouper étiquettes proches.
- Pseudo-labeling & self-training (teacher → student), puis réentrainement sur tout le dataset.
- Data augmentation textuelle (back-translation, synonym replace) si viable.

**HYPERPARAMÈTRES**
- Sentiment : `k` (SelectKBest chi2), `C` (LR), optionnel `n_components` (SVD).
- Emotions : `C` (LR), seuils par label (`thr`), régularisation `alpha` si SGD.
- Stratégies : GridSearch / RandomizedSearch, K-fold pour thresholds, calibration (sigmoid).

**INFRASTRUCTURE**
- GPU pour SBERT encoding (batch large).
- Memmap + partial_fit (SGD) pour entraîner sur tout le jeu si RAM limitée.
- Calibration/OneVsRest en CPU si GPU pas nécessaire.
- Logging et checkpoints pour reprendre si plantage.

# Optimisation — méthode et objectifs

**Objectif** : améliorer les performances des modèles *Sentiment* et *Emotions* (AP, F1 micro/macro) en agissant sur :
- **DATA** : cleaner / équilibrer / augmenter (ici : pseudo-labels, filtrage des labels rares).
- **HYPERPARAMÈTRES** : C pour LogisticRegression, alpha pour SGD, learning_rate, etc.
- **INFRA / LIBS** : SBERT embeddings (GPU), memmap + partial_fit (RAM limite), calibration (CalibratedClassifierCV).
- **MÉTRIQUES & COMPARAISON** : AP par label (average_precision), F1 micro/macro, tableau comparatif.

Approche :
1. Diagnostiquer états & jeux de données existants.
2. Construire / réutiliser pipelines pour sentiment et emotions.
3. Faire une **recherche rapide d’hyperparamètres** sur un *sous-échantillon* (pour ne pas surcharger la machine).
4. Si positif, entraîner la version finale (sur train+val) — en mode mémoire-safe (memmap & SGD partial_fit pour large dataset).
5. Calibrer probabilités, trouver thresholds (K-fold thresholds) et évaluer sur test.
6. Sauvegarder artefacts et produire un tableau de comparaison.

In [3]:
# Diagnostics rapides — détecte objets existants et capacités machine
import os, psutil, joblib, numpy as np, pandas as pd, gc
print("Répertoire artifacts existe :", os.path.exists("artifacts"))
print("Fichiers artifacts (quelques-uns) :", sorted(os.listdir("artifacts"))[:30] if os.path.exists("artifacts") else [])
print("Objets en mémoire (quelques clés) :", [k for k in globals().keys() if k.lower().startswith(("emo","sent","xtr","xte","ytr","yte","clf"))][:60])
print("RAM totale (GB):", psutil.virtual_memory().total/1024**3, " | disponible (GB):", psutil.virtual_memory().available/1024**3)
gc.collect()

Répertoire artifacts existe : True
Fichiers artifacts (quelques-uns) : ['X_all_memmap.dat', 'X_all_memmap_meta.joblib', 'absa_auto_ensemble_conservative.csv', 'absa_auto_ensemble_conservative_disagreements.csv', 'absa_auto_ensemble_disagreements.csv', 'absa_auto_ensemble_labels.csv', 'absa_auto_ensemble_ml_disagreements.csv', 'absa_auto_ensemble_ml_labels.csv', 'absa_auto_mapping.joblib', 'absa_candidates_embeddings.joblib', 'absa_candidates_embs_reduced_labels.joblib', 'absa_candidates_umap_embs_labels.joblib', 'absa_manual_qc_sample.csv', 'absa_reviews_with_auto_aspects.csv', 'emo_calibrated_ovr_sigmoid.joblib', 'emo_clf_final_trainval.joblib', 'emo_clf_final_trainval_calibrated.joblib', 'emo_partial_test_artifacts.joblib', 'emo_sbert_scaler.joblib', 'emo_sgd_partial_models.joblib', 'emo_sgd_partial_models_epoch1.joblib', 'emo_sgd_partial_models_epoch10.joblib', 'emo_sgd_partial_models_epoch11.joblib', 'emo_sgd_partial_models_epoch12.joblib', 'emo_sgd_partial_models_epoch13.joblib', 

0

## Optimisation - Sentiment

Stratégie:
- Diagnostic des jeux (texts / y) et du modèle existant (recycler si présent).
- Baseline rapide (réentrainement sur petit échantillon ou ré-évaluation du modèle existant).
- Recherche d'hyperparamètres légère (C pour LR) sur sous-échantillon pour ne pas saturer la RAM.
- Si dataset grand -> option memmap + SGDClassifier (partial_fit).
- Mesures : accuracy / AP (si multi), F1 micro/macro, matrice de confusion.

In [17]:
# Imports & chemins
from pathlib import Path
import time, json
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,
                             average_precision_score, precision_recall_curve, roc_curve,
                             f1_score, PrecisionRecallDisplay)
from sklearn.calibration import CalibratedClassifierCV
from joblib import load, dump
from scipy import sparse
import matplotlib.pyplot as plt

RNG = 42
ART_DIR = Path("models")             # là où tu as sauvegardé sel / sfm / svd_pipe
DATA_DIR = Path("data/processed")    # là où sont X_tfidf_*.npz et y_*.joblib si existants
OUT_DIR = Path("models"); OUT_DIR.mkdir(parents=True, exist_ok=True)

# Trouver les derniers X/y produits en 5.2.1/5.2.2
X_CAND = sorted(DATA_DIR.glob("X_tfidf*.npz"))
Y_CAND = sorted(DATA_DIR.glob("y_*binary*.joblib"))
print("X dispo:", [p.name for p in X_CAND][-3:])
print("y dispo:", [p.name for p in Y_CAND][-3:])

X dispo: ['X_tfidf_sample.npz']
y dispo: ['y_binary_sample.joblib']


In [18]:
assert X_CAND and Y_CAND, "Pas de features X / labels y trouvés. Exécute 5.2.1 pour les générer."
X_path = X_CAND[-1]; y_path = Y_CAND[-1]
print("Chargement:", X_path.name, "|", y_path.name)

X_all = sparse.load_npz(X_path)
y_all = load(y_path).astype(int)
X_all.shape, y_all.shape

Chargement: X_tfidf_sample.npz | y_binary_sample.joblib


((120000, 180007), (120000,))

In [20]:
# 80/10/10 (ou adapte selon tes besoins)
X_tr, X_tmp, y_tr, y_tmp = train_test_split(
    X_all, y_all, test_size=0.20, stratify=y_all, random_state=RNG
)
X_va, X_te, y_va, y_te = train_test_split(
    X_tmp, y_tmp, test_size=0.50, stratify=y_tmp, random_state=RNG
)
print("train:", X_tr.shape, "| val:", X_va.shape, "| test:", X_te.shape, "| pos ratio train:", y_tr.mean())

train: (96000, 180007) | val: (12000, 180007) | test: (12000, 180007) | pos ratio train: 0.7614583333333333


In [21]:
# === RESCUE CELL ===
# Reconstruit X_tr_, X_va_, X_te_ à partir de X_tr/X_va/X_te en appliquant ton pipeline (chi2 -> L1 -> SVD si présents)
import numpy as np
from scipy import sparse

def _apply_optional(sel_obj, X):
    return sel_obj.transform(X) if sel_obj is not None else X

# 1) Vérifs d'existence des bases
assert 'X_tr' in globals() and 'X_va' in globals() and 'X_te' in globals(), \
    "Il faut d'abord exécuter la cellule qui split X_all/y_all en X_tr, X_va, X_te."

# 2) Appliquer EXACTEMENT ce que tu as prévu :
#    d'abord chi² si 'sel' est présent, puis L1 SFM si 'sfm', puis SVD si 'svd_pipe'
X_tr_ = X_tr
X_va_ = X_va
X_te_ = X_te

# chi²
if 'sel' in globals() and sel is not None:
    X_tr_ = sel.transform(X_tr_)
    X_va_ = sel.transform(X_va_)
    X_te_ = sel.transform(X_te_)
    print("[OK] chi² appliqué ->", X_tr_.shape)

# L1 SFM (optionnel)
if 'sfm' in globals() and sfm is not None:
    X_tr_ = sfm.transform(X_tr_)
    X_va_ = sfm.transform(X_va_)
    X_te_ = sfm.transform(X_te_)
    print("[OK] L1 SFM appliqué ->", X_tr_.shape)

# SVD (optionnel) — produit souvent des denses
if 'svd_pipe' in globals() and svd_pipe is not None:
    # svd_pipe peut être un Pipeline (ex: Normalizer + TruncatedSVD)
    X_tr_ = svd_pipe.transform(X_tr_)
    X_va_ = svd_pipe.transform(X_va_)
    X_te_ = svd_pipe.transform(X_te_)
    # Assure un dtype float32
    X_tr_ = np.asarray(X_tr_, dtype=np.float32)
    X_va_ = np.asarray(X_va_, dtype=np.float32)
    X_te_ = np.asarray(X_te_, dtype=np.float32)
    print("[OK] SVD appliqué ->", X_tr_.shape, "(dense)")

# 3) Mise au bon format mémoire
# - si c’est encore sparse, garde CSR float32 (cohérent avec tes cellules)
# - si c’est dense (après SVD), force np.float32
def _to_ideal_format(X):
    if sparse.issparse(X):
        return X.tocsr().astype(np.float32)
    return np.asarray(X, dtype=np.float32)

X_tr_ = _to_ideal_format(X_tr_)
X_va_ = _to_ideal_format(X_va_)
X_te_ = _to_ideal_format(X_te_)

# 4) y en int8
assert 'y_tr' in globals() and 'y_va' in globals() and 'y_te' in globals(), \
    "Il faut que y_tr/y_va/y_te existent (créés lors du split)."
y_tr = np.asarray(y_tr, dtype=np.int8)
y_va = np.asarray(y_va, dtype=np.int8)
y_te = np.asarray(y_te, dtype=np.int8)

# 5) Logs utiles
def _dens(X):
    if sparse.issparse(X):
        return X.nnz / (X.shape[0]*X.shape[1])
    return np.count_nonzero(X) / (X.size + 1e-9)

print(f"[SHAPES] train {X_tr_.shape} | val {X_va_.shape} | test {X_te_.shape}")
print(f"[DENSITY] train {(_dens(X_tr_))*100:.3f}% | val {(_dens(X_va_))*100:.3f}% | test {(_dens(X_te_))*100:.3f}%")
print("[DONE] X_tr_, X_va_, X_te_ prêts.")


[SHAPES] train (96000, 180007) | val (12000, 180007) | test (12000, 180007)
[DENSITY] train 0.316% | val 0.314% | test 0.322%
[DONE] X_tr_, X_va_, X_te_ prêts.


In [22]:
# === C5.2.4 — GRIDSEARCH ÉLARGI (sur tes features existantes X_tr_, X_va_) ===
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import average_precision_score, f1_score
import numpy as np, pandas as pd, time, joblib
from scipy import sparse

# 0) sécurité formats mémoire (tu l'avais déjà fait mais on blind)
X_tr_ = X_tr_.tocsr().astype(np.float32)
X_va_ = X_va_.tocsr().astype(np.float32)
X_te_ = X_te_.tocsr().astype(np.float32)
y_tr  = np.asarray(y_tr, dtype=np.int8)
y_va  = np.asarray(y_va, dtype=np.int8)
y_te  = np.asarray(y_te, dtype=np.int8)

# 1) grille plus large (lbfgs vs saga, C, class_weight)
base = LogisticRegression(max_iter=5000, tol=1e-3)
param_grid = {
    "solver": ["lbfgs", "saga"],         # saga gère des cas + L1, mais on garde l2 pour stabilité
    "penalty": ["l2"],
    "C": [0.1, 0.25, 0.5, 1.0, 2.0, 5.0],
    "class_weight": [None, "balanced"]
}
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RNG)

# 2) GridSearch sur TRAIN uniquement (on garde VAL pour sélection seuil & calibration)
gs = GridSearchCV(
    estimator=base,
    param_grid=param_grid,
    scoring="average_precision",   # cohérent avec ton AP
    cv=cv,
    n_jobs=1,                      # éviter copies mémoire massives
    verbose=1,
    refit=True
)

t0 = time.time()
gs.fit(X_tr_, y_tr)
t_gs = time.time() - t0
print(f"[Grid] best={gs.best_params_} | best AP(cv)={gs.best_score_:.4f} | time={t_gs:.1f}s")

# 3) Éval rapide sur VAL (avant calibration)
proba_va_gs = gs.best_estimator_.predict_proba(X_va_)[:,1]
ap_va_gs = average_precision_score(y_va, proba_va_gs)
print(f"[Grid] AP(val) non calibré = {ap_va_gs:.4f}")

# 4) Enregistrer la grille (preuves C5.2.4)
joblib.dump(gs, MODELS_DIR/"grid_sentiment_logreg.joblib")
print("Saved:", (MODELS_DIR/"grid_sentiment_logreg.joblib").as_posix())

Fitting 3 folds for each of 24 candidates, totalling 72 fits


c:\Users\antoi\OneDrive\Documents\Ynov\Projet fil rouge\Bloc 5\amazon-reviews-insights\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\antoi\OneDrive\Documents\Ynov\Projet fil rouge\Bloc 5\amazon-reviews-insights\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_

[Grid] best={'C': 1.0, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'lbfgs'} | best AP(cv)=0.9838 | time=21812.6s
[Grid] AP(val) non calibré = 0.9832


NameError: name 'MODELS_DIR' is not defined

In [23]:
# 4) Enregistrer la grille (preuves C5.2.4)
joblib.dump(gs, OUT_DIR/"grid_sentiment_logreg.joblib")
print("Saved:", (OUT_DIR/"grid_sentiment_logreg.joblib").as_posix())

Saved: models/grid_sentiment_logreg.joblib
